<a href="https://colab.research.google.com/github/eminekilics/recruitment_bias-fairness_ai/blob/main/recruitment_bias%26fairness_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

loading the dataset

In [ ]:
df = pd.read_csv("dataset.csv")

In [ ]:
print("Veri boyutu:", df.shape)
print(df.head())

Veri boyutu: (2000, 6)
   gender  age education_level  experience_years  screening_score  shortlisted
0    Male   35         Masters                 1        72.473130            0
1  Female   32       Bachelors                15        55.004084            0
2    Male   36         Masters                19        60.156004            0
3    Male   44       Bachelors                 6        75.846397            0
4    Male   39     High School                 1        43.297540            1


 encode categorical variables

In [ ]:
le = LabelEncoder()
df["gender"] = le.fit_transform(df["gender"])   # Male=1, Female=0
df["education_level"] = le.fit_transform(df["education_level"])

feature and target separation

In [ ]:
X = df.drop("shortlisted", axis=1)
y = df["shortlisted"]

scaling

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

splitting training/test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

build and train the model

In [37]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

predicting

In [ ]:
y_pred = model.predict(X_test)

print("Genel Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Genel Accuracy: 0.5975
              precision    recall  f1-score   support

           0       0.60      1.00      0.75       239
           1       0.00      0.00      0.00       161

    accuracy                           0.60       400
   macro avg       0.30      0.50      0.37       400
weighted avg       0.36      0.60      0.45       400



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


gender-based success rate (fairness analysis)

In [ ]:
df_test = pd.DataFrame(X_test, columns=df.drop("shortlisted", axis=1).columns)
df_test["gender"] = df.loc[y_test.index, "gender"].values
df_test["y_true"] = y_test.values
df_test["y_pred"] = y_pred

Separate accuracy for men and women

In [ ]:
male_acc = accuracy_score(df_test[df_test["gender"] == 1]["y_true"],
                          df_test[df_test["gender"] == 1]["y_pred"])
female_acc = accuracy_score(df_test[df_test["gender"] == 0]["y_true"],
                            df_test[df_test["gender"] == 0]["y_pred"])

print("accuracy for male candidates:", male_acc)
print("accuracy for female candidates:", female_acc)

accuracy for male candidates: 0.583756345177665
accuracy for female candidates: 0.6108374384236454


checking for bias

In [ ]:
if abs(male_acc - female_acc) > 0.1:
    print("There is potential bias in the model, with a significant difference between genders.")
else:
    print("No significant bias was observed in the model.")

No significant bias was observed in the model.
